In [1]:
import os
import fiona
from shapely.geometry import shape, mapping, box
from rasterio.warp import reproject, Resampling
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from rasterio.merge import merge
import glob
import numpy as np
from scipy.ndimage import gaussian_filter

In [2]:
rootdirectory='big_tree/prediciton/'
os.chdir(rootdirectory)

In [3]:
def get_extent(src):
    bounds = src.bounds
    extent = (bounds.left, bounds.bottom, bounds.right, bounds.top)
    return extent

In [4]:
#Resample Mask to match sentinel data
def resample_to_match(src,band_data, ref_raster_path, dst_path):
    with rasterio.open(ref_raster_path) as ref:
        ref_transform = ref.transform
        ref_width = ref.width
        ref_height = ref.height
        ref_crs = ref.crs
        
        kwargs = src.meta.copy()
        kwargs.update({
            'count': 2,
            'crs': ref_crs,
            'transform': ref_transform,
            'width': ref_width,
            'height': ref_height
        })

        with rasterio.open(dst_path, 'w', **kwargs) as dst:
            reproject(
                source=band_data,
                destination=rasterio.band(dst, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=Resampling.nearest
            )
            print(f"Mask save to {dst_path}")


In [7]:
# Filter specific tile identifiers
numvar = ['009','010', '011', '012', '020', '021', '022', '023', '024', 
          '031', '032', '033', '034', '035', '036', '042', '043',  
          '044', '045', '046', '047', '048', '054', '055', '056', 
          '057', '058', '059', '060', '065', '066', '067', '068', 
          '069', '070', '071', '072', '076', '077', '078', '079', 
          '080', '081', '082', '083', '084', '087', '088', '089', 
          '090', '091', '092', '093', '094', '095', '096', '099', 
          '100', '101', '102', '103', '104', '105', '106', '107', 
          '108', '109', '110', '111', '112', '113', '114', '115', 
          '116', '117', '118', '119', '120', '121', '122', '123', 
          '124', '125', '126', '127', '128', '129', '130', '131', '132']

In [8]:
output_dir = os.path.join(rootdirectory,"pred")
os.makedirs(output_dir, exist_ok=True)

In [9]:
for i in numvar:  # Assuming 4x4 tiles
    src_path = f'prediction_{i}.tif'
    with rasterio.open(src_path) as src:
        band_data = src.read()
    # Open each tile image
    ref_raster_path = f'Sentinel_{i}.tif'
    print(f"Opening {ref_raster_path}")
    
    # Write the cropped image to a new file
    dst_path = f'pred/prediction_{i}.tif'

    #Crop Mask to Image extent so they align perfectly
    resample_to_match(src,band_data,ref_raster_path, dst_path)

Opening G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3/Images/Sentinel_012.tif
Mask save to G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3/croppedpredGLOBRMSE/prediction_012.tif
Opening G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3/Images/Sentinel_022.tif
Mask save to G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3/croppedpredGLOBRMSE/prediction_022.tif
Opening G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3/Images/Sentinel_023.tif
Mask save to G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3/croppedpredGLOBRMSE/prediction_023.tif
Opening G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3/Images/Sentinel_031.tif
Mask save to G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3/croppedpredGLOBRMSE/prediction_031.tif
Opening G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3/Images/Sentinel_042.tif
Mask save to G:/Conservation Solution 

In [8]:
output_dir

'G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\\croppedpredENS_WGL'

In [10]:
input_folder = output_dir#os.path.join(rootdirectory, "croppedpredENS/")
output_path = os.path.join(input_folder, 'Canopy_Height_ENS_Unc.tif')

# Pattern for finding specific tiles
tile_pattern = os.path.join(input_folder, 'prediction_*.tif')
tile_paths = glob.glob(tile_pattern)

# Filter specific tile identifiers
numvar = ['009','010', '011', '012', '020', '021', '022', '023', '024', 
          '031', '032', '033', '034', '035', '036', '042', '043',  
          '044', '045', '046', '047', '048', '054', '055', '056', 
          '057', '058', '059', '060', '065', '066', '067', '068', 
          '069', '070', '071', '072', '076', '077', '078', '079', 
          '080', '081', '082', '083', '084', '087', '088', '089', 
          '090', '091', '092', '093', '094', '095', '096', '099', 
          '100', '101', '102', '103', '104', '105', '106', '107', 
          '108', '109', '110', '111', '112', '113', '114', '115', 
          '116', '117', '118', '119', '120', '121', '122', '123', 
          '124', '125', '126', '127', '128', '129', '130', '131', '132']
filtered_tile_paths = [tile_path for tile_path in tile_paths if any(num in tile_path for num in numvar)]

# Print the list of filtered tile file paths
print("Filtered tile file paths:")
for tile_path in filtered_tile_paths:
    print(tile_path)

Filtered tile file paths:
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_009.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_010.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_011.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_012.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_020.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_021.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_022.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_023.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_024.tif
G:/Conservation Solution Lab/People/Lu

In [11]:
input_folder = output_dir#os.path.join(rootdirectory, "croppedpredENS/")
output_path = os.path.join(input_folder, 'Canopy_Height_ENS_Unc.tif')

# Pattern for finding specific tiles
tile_pattern = os.path.join(input_folder, 'prediction_*.tif')
tile_paths = glob.glob(tile_pattern)

# Filter specific tile identifiers
numvar = ['009','010', '011', '012', '020', '021', '022', '023', '024', 
          '031', '032', '033', '034', '035', '036', '042', '043',  
          '044', '045', '046', '047', '048', '054', '055', '056', 
          '057', '058', '059', '060', '065', '066', '067', '068', 
          '069', '070', '071', '072', '076', '077', '078', '079', 
          '080', '081', '082', '083', '084', '087', '088', '089', 
          '090', '091', '092', '093', '094', '095', '096', '099', 
          '100', '101', '102', '103', '104', '105', '106', '107', 
          '108', '109', '110', '111', '112', '113', '114', '115', 
          '116', '117', '118', '119', '120', '121', '122', '123', 
          '124', '125', '126', '127', '128', '129', '130', '131', '132']
filtered_tile_paths = [tile_path for tile_path in tile_paths if any(num in tile_path for num in numvar)]

# Print the list of filtered tile file paths
print("Filtered tile file paths:")
for tile_path in filtered_tile_paths:
    print(tile_path)

# Initialize list for rasterio open files and get metadata from the first file
imagens_mosaico = []
for fp in filtered_tile_paths:
    src = rasterio.open(fp)
    imagens_mosaico.append(src)
    
    # Copy metadata from the first image for output
    if 'out_meta' not in locals():
        out_meta = src.meta.copy()
        original_crs = src.crs

# Merge tiles using the 'sum' method to accumulate values
mosaico_sum, out_trans = merge(imagens_mosaico, method='sum')
print(f'Mosaic sum done. shape: {mosaico_sum.shape}')

# Merge tiles using the 'count' method to count valid pixels in each location
mosaico_count, _ = merge(imagens_mosaico, method='count')
print(f'Mosaic count done. Shape: {mosaico_count.shape}')

# Avoid division by zero by setting count of zero pixels to NaN
with np.errstate(divide='ignore', invalid='ignore'):
    mosaico_avg = mosaico_sum / np.where(mosaico_count == 0, np.nan, mosaico_count)

print(f'Mosaic average done')

# # Apply Gaussian smoothing (adjust sigma for different smoothing levels)
# sigma = 0.1  # Standard deviation for Gaussian kernel; adjust as needed
# mosaico_smoothed = np.where(
#     ~nan_mask,  # Only apply smoothing where data exists
#     gaussian_filter(mosaico_avg, sigma=sigma),
#     mosaico_avg  # Keep nodata values unchanged
# )
mosaico_smoothed = mosaico_avg 
print(f'Smoothed shape: {mosaico_smoothed.shape}')

# Ensure correct shape: should be (1, height, width) for a single-band output
# Ensure the shape is (height, width) before writing
if mosaico_smoothed.ndim == 4:
    mosaico_smoothed = np.squeeze(mosaico_smoothed, axis=0)  # Remove the first axis if it is of size 1


# Final confirmation printout for debugging
print(f"Final shape of mosaico_smoothed: {mosaico_smoothed.shape}")



Filtered tile file paths:
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_009.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_010.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_011.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_012.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_020.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_021.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_022.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_023.tif
G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\prediction_024.tif
G:/Conservation Solution Lab/People/Lu

In [12]:
# Update metadata for a single-band output using mosaico_smoothed dimensions
out_meta.update({
    'count': mosaico_smoothed.shape[0],  # Number of bands
    'height': mosaico_smoothed.shape[1],  # Corresponds to height (y-dimension)
    'width': mosaico_smoothed.shape[2],   # Corresponds to width (x-dimension)
    'transform': out_trans,
    'crs': original_crs,  # Ensure CRS matches the first image
    'nodata': -999.9,
    'dtype': 'float32'
})
print(f'Saving ...')

# Write the predicted numpy array as a GeoTIFF
with rasterio.open(output_path, 'w', **out_meta) as dest:
    dest.write(mosaico_smoothed)  # Write as a single band

# Close all source files
for src in imagens_mosaico:
    src.close()

print(f'Saved smoothed prediction to {output_path}')

Saving ...
Saved smoothed prediction to G:/Conservation Solution Lab/People/Luizmar/PhD_Luizmar/Chapter_3\croppedpredENS_WGL\Canopy_Height_ENS_Unc.tif


In [13]:
import gc
gc.collect()

0